<a href="https://colab.research.google.com/github/MajoCarb/AmphoraProject/blob/master/main_EUR_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AMPHORA CHALLENGE
#### By María José Carbajal Alonso

In [46]:
#importar librerias
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
#Ancestry file
ancestry_file = pd.read_csv('drive/MyDrive/Amphora/ancestry-1000genome.tsv', sep='\t')
#ancestry_file = pd.read_csv('ancestry-1000genome.tsv', sep='\t',index_col='Sample name')

#Superpopulation code dataframe
spp_code = pd.DataFrame()
spp_code['sample_name'] = ancestry_file['Sample name']

#getting the patients only
patients = spp_code['sample_name']

spp_code['superpopulation_code'] = ancestry_file['Superpopulation code']
spp_code.set_index('sample_name', inplace=True)

#getting the 5 categories we will use (ancestry types)
ancestry_types = list(spp_code['superpopulation_code'].unique())

spp_code

AFR
AFRAFR
AFRAFRAFR
AFRAFRAFRAFR
AFRAFRAFRAFRAFR
AFRAFRAFRAFRAFRAFR
AFRAFRAFRAFRAFRAFRAFR
AFRAFRAFRAFRAFRAFRAFRAFR
AFRAFRAFRAFRAFRAFRAFRAFRAFR
AFRAFRAFRAFRAFRAFRAFRAFRAFRAFR


,superpopulation_code
sample_name,
NA19625,AFR
NA19835,AFR
NA19900,AFR
NA19917,AFR
NA19703,AFR
...,...
NA19201,AFR
NA19206,AFR
NA19213,AFR


In [180]:
#Reduction of number of patients using a sample of 100%, random state makes that the sample be the same 
#using 60% of data, takes 50 minutes to create de dataframe with the ancestry

patients_=patients.sample(frac=1, random_state=1)
list(patients_.head(1).values)[0]
print(str(np.array(patients_[:5])))
len(patients_)

['NA20876' 'HG01148' 'HG01366' 'HG01992' 'HG01851']


2504

In [181]:
#selecting some patients in order to reduce data
# patients_ = patients[426:449]
# patients_= patients_.append([patients[524:542], patients[920:938], patients[1028:1046]])
main_df = pd.DataFrame()
lenn= 10028 #total len = 10028 of chr:location list

for patient in patients_:   


    patient_x = pd.read_csv('drive/MyDrive/Amphora/Challenge_Samples/'+patient+'.csv')
    patient_x.rename(columns={patient:'Presence'},inplace=True)
    
    #Creating the ALT and Presence lists (These will be the columns)    
    alt_list= list(patient_x['ALT'])
    alt_list=alt_list[:lenn]
    presence_list = list(patient_x['Presence'])
    presence_list=presence_list[:lenn]

    
    sequence = ("".join(map(str, presence_list)))
    code = list(spp_code[spp_code.index==patient]['superpopulation_code'] )
    code

    afr_res = 1  if code[0] == 'AFR' else 0
    eur_res = 1  if code[0] == 'EUR' else 0
    asian_res = 1  if code[0] == 'SAS' or code[0] == 'EAS' else 0

    data = [patient, code[0], sequence, afr_res, eur_res, asian_res]
    col = ['Patient', 'SPP_code', 'Sequence', 'AFR', 'EUR','ASIAN']

    sample = {'Patient':patient, 'SPP_code':code[0], 'Sequence':sequence, 'AFR':afr_res, 'EUR': eur_res,'ASIAN':asian_res}
    data

    df = pd.DataFrame(columns=col)
    df=df.append(sample, ignore_index=True)
    df.set_index('Patient')

    main_df = pd.concat([main_df,df], axis=0)

    

#patients_main_df=list(set(main_df.index.get_level_values('Patient')))
#spp_code_main_df=list(set(main_df.index.get_level_values('Superpopulation Code')))
main_df=main_df.set_index('Patient')



In [161]:
main_df

,SPP_code,Sequence,AFR,EUR,ASIAN
Patient,,,,,
NA20876,SAS,0000101011011010111011100010000010100011101000...,0,0,1
HG01148,AMR,0110100011011010111010000110000010100010110000...,0,0,0
HG01366,AMR,0011100111011010111011000010000011100000110000...,0,0,0
HG01992,AMR,0110100011111010111011000010001011101001101010...,0,0,0
HG01851,EAS,0001100000011000111011000010000011000101101000...,0,0,1
...,...,...,...,...,...
HG03971,SAS,0101110011101011111000000100000010100101101000...,0,0,1
HG01259,AMR,0100100001111010111011100000000010100011111000...,0,0,0
NA19448,AFR,0010000001111010110010100110000000101100100000...,1,0,0


In [182]:
print(main_df['ASIAN'].sum(),
main_df['AFR'].sum(),
main_df['EUR'].sum())

993 661 503


In [125]:
#main_df.xs('EUR',level='Superpopulation Code')
main_df.iloc[main_df.index.get_loc('NA19681')][1]
# main_df.index.names
#spp_code_main_df
#patients_main_df[:5]

'0'

In [ ]:
main_df
ancestry_freq=main_df.index.get_level_values('Superpopulation Code').value_counts()//10028
ancestry_freq

In [27]:
list(main_df.index.get_level_values('Superpopulation Code').unique())

['SAS', 'AMR', 'EAS', 'AFR', 'EUR']

In [184]:
#Saving the datafram into a CSV file at Google Drive
main_df.to_csv('drive/MyDrive/Amphora/main_df_100%_seq.csv', index=True)
df_without_index = pd.read_csv('drive/MyDrive/Amphora/main_df_100%_seq.csv')
df_without_index
main_df = df_without_index
#main_df.set_index()

In [185]:
main_df

,Patient,SPP_code,Sequence,AFR,EUR,ASIAN
0,NA20876,SAS,0000101011011010111011100010000010100011101000...,0,0,1
1,HG01148,AMR,0110100011011010111010000110000010100010110000...,0,0,0
2,HG01366,AMR,0011100111011010111011000010000011100000110000...,0,0,0
3,HG01992,AMR,0110100011111010111011000010001011101001101010...,0,0,0
4,HG01851,EAS,0001100000011000111011000010000011000101101000...,0,0,1
...,...,...,...,...,...,...
2499,NA20901,SAS,0101110011011010110001100110000011110111111000...,0,0,1
2500,HG02861,AFR,0011100001111110101000100110000011001011110000...,1,0,0
2501,NA18637,EAS,0100100000011000111011000010000001000101101000...,0,0,1
2502,HG03809,SAS,0101110011001010111011100010000011110011111000...,0,0,1


## Here starts the section of training the model

In [186]:
from sklearn.model_selection import train_test_split
import numpy as np


##**European ancestry KNN** 

In [208]:
#Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
import numpy as np

In [189]:
#drop the ancestries we are not going to predict
df_eur = main_df.drop(['ASIAN', 'AFR'], axis=1)

#df_eur = df_eur[df_eur['EUR']==1]

df_eur.set_index(['Patient'],inplace=True)
#df_eur
#len((set(df_eur.index.get_level_values('Patient'))))
patients_eur_df=list(set(df_eur.index.get_level_values('Patient')))
#len(patients_eur_df)

df_eur

,SPP_code,Sequence,EUR
Patient,,,
NA20876,SAS,0000101011011010111011100010000010100011101000...,0
HG01148,AMR,0110100011011010111010000110000010100010110000...,0
HG01366,AMR,0011100111011010111011000010000011100000110000...,0
HG01992,AMR,0110100011111010111011000010001011101001101010...,0
HG01851,EAS,0001100000011000111011000010000011000101101000...,0
...,...,...,...
NA20901,SAS,0101110011011010110001100110000011110111111000...,0
HG02861,AFR,0011100001111110101000100110000011001011110000...,0
NA18637,EAS,0100100000011000111011000010000001000101101000...,0


In [ ]:
# Import LabelEncoder
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
seq_encoded=le.fit_transform(df_eur['Sequence'])

print(set(list(seq_encoded)))
type(seq_encoded)
list(le.classes_)

In [ ]:
x= seq_encoded
x1D = seq_encoded
x = x.reshape(-1, 1)
y = df_eur[['EUR']]
y = y.values


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8,random_state=1)
#X_train.sort_values(by='Patient')
y_test




In [230]:
from sklearn.neighbors import KNeighborsClassifier
#model not for CV
knn_model = KNeighborsClassifier(n_neighbors=2)

#create a new KNN model for CV
knn_cv = KNeighborsClassifier(n_neighbors=2)
cv_scores = cross_val_score(knn_cv, X_train,np.ravel(y_train), cv=10,scoring='accuracy')

#training the model
knn_model.fit(X_train,np.ravel(y_train))

# Use the knn_model's predict method on the test data
y_pred = knn_model.predict(X_test)
y_pred

# print the first 10 predicted probabilities of class membership
y_pred_prob=knn_model.predict_proba(X_test)
print("Predicted probabilities",len(y_pred_prob))

Predicted probabilities 501


In [225]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Use the forest's predict method on the test data

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))
#print each cv score (accuracy) and average them
#print(cv_scores)

print('Acuracy cv_scores mean:{}'.format(np.mean(cv_scores)))

Accuracy: 0.7904191616766467
Precision: 0.5405405405405406
Recall: 0.18518518518518517
Predicted probabilities [[0.5 0.5]
 [1.  0. ]
 [1.  0. ]
 [0.5 0.5]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [0.5 0.5]
 [0.  1. ]
 [1.  0. ]]
Acuracy cv_scores mean:0.8002835820895522


In [205]:
#IMPORTANT: first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
print(metrics.confusion_matrix(y_test, y_pred))

[[376  17]
 [ 88  20]]


In [195]:
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

[[376  17]
 [ 88  20]]


In [196]:
# use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred))

0.7904191616766467
0.7904191616766467


In [197]:
#Sensitivity: When the actual value is positive, how often is the prediction correct?
sensitivity = TP / float(FN + TP)
print(sensitivity)
print(metrics.recall_score(y_test, y_pred))

0.18518518518518517
0.18518518518518517


In [198]:
precision = TP / float(TP + FP)

print(precision)
print(metrics.precision_score(y_test, y_pred))

0.5405405405405406
0.5405405405405406


In [251]:
#AUC

# IMPORTANT: first argument is true values, second argument is predicted probabilities
#print(metrics.roc_auc_score(y_test, y_pred_prob))

print("cross-validated AUC: ",cross_val_score(knn_cv, X_train,np.ravel(y_train), cv=10,scoring='roc_auc').mean())


cross-validated AUC:  0.7024313883580188


## **African ancestry logistic**